# Capstone Project Course Assignment

## Part 1 : Creating the data frame of postal codes, Boroughs and Neighborhoods

In [1]:
import pandas as pd

In [2]:
# url of the wikipedia page
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#reading the page content with pandas
d=pd.read_html(url)

### Let's check the type of d

In [4]:
type(d)

list

## Let's find how many elements we have in ths list and their type

In [5]:
print('Number of elements : ', len(d))
print('Type : ', type(d[0]))

Number of elements :  3
Type :  <class 'pandas.core.frame.DataFrame'>


## So, we have extracted a list of 3 data frames. The first data frame contain the table of interest

In [6]:
d[0].head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Let's create our dataframe df that will contain the data of interest

In [7]:
df=d[0]
print('Shape of the dataframe : ', df.shape)
df.head()

Shape of the dataframe :  (180, 3)


,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Droping the rows that Neighborhood are 'Not assigned'

#### Let's first check how many rows have the value 'Not assigned' on the Borough column

In [8]:
len(df[df['Borough']=='Not assigned'].index)

77

#### Droping these 77 rows

In [9]:
#Create a list of the index of all the raws with the value of 'Not assigned' on the 'Borough' feature
Not_assigned_indexes=df[df['Borough']=='Not assigned'].index

#Droping the raws corresponding to these indexes
df.drop(Not_assigned_indexes, axis=0, inplace=True)

#### Let's vaidate that all raws having 'Not assigned' on the Borough column are removed

In [10]:
len(df[df['Borough']=='Not assigned'].index)

0

So, all raws with a 'Not assigned' Borough were removed 

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

#### Let's first check if there are raws with a 'Not assigned' neighborhood

In [11]:
len(df[df['Neighborhood']=='Not assigned'].index)

0

So, there is no raws having a Not assigned Neighborhood

## Part 2 : adding latitude and logitudeto the data frame

In [12]:
#Reading the csv file of the latitude and logitude
df2=pd.read_csv('http://cocl.us/Geospatial_data')

#Let's have a look on the dataframe
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


### Now,we will merge the 2 data frames in order to get the 5 columns (Borougn, Neighborhood, Postal code, Latitude, Logitue) on the same data frame

In [13]:
#Using the merge function of pandas
df_Toronto=pd.merge(df,df2)

print('Shape of the data frame : ', df_Toronto.shape)

#Having a look on the new dataframe version 
df_Toronto.head(10)

Shape of the data frame :  (103, 5)


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


# Part 3 : Explore and cluster the neighborhoods in Toronto

## Let's consider only the neighborhoods of North York borough

#### Coordinates of North York

In [14]:
from geopy.geocoders import Nominatim
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library
import requests

#### Creating a sub data frame for North York Borough

In [15]:
North_York_data = df_Toronto[df_Toronto['Borough'] == 'North York'].reset_index(drop=True)
North_York_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
3,M3B,North York,Don Mills,43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [16]:
address = 'North York, Toronto, CA'

geolocator = Nominatim(user_agent="North_York_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York are 43.7543263, -79.44911696639593.


In [17]:
# create map of North York using latitude and longitude values
map_North_York = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_North_York)  
    
map_North_York

### Exploring the neighborhoods of North York usinf Foursquare

#### Defining Foursquare Credentials

In [18]:
CLIENT_ID = 'WCO3PQS02C5V3FHVMAGTI5N2TNM5UBSKGZYQYRLOPEAGESXB' # your Foursquare ID
CLIENT_SECRET = 'MHHHHT2OT5TZFBMRVV4QK3NBX51RGSEKJTWLWBMNNX1LWYV4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#### Let's pick a neighborhood in the North York Borough and let it be the second we have in the North York data set

In [19]:
print('The first neighborhood we get in North York is : ', North_York_data.loc[1,'Neighborhood'])

The first neighborhood we get in North York is :  Victoria Village


#### Getting latitude and logitude of Victoria Village

In [20]:
neighborhood_latitude = North_York_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = North_York_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name = North_York_data.loc[1, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Victoria Village are 43.725882299999995, -79.31557159999998.


#### Let's explore the first 80 venues of Victoria Village within a raidus of 400 meters

Creating the url first

In [21]:
LIMIT = 80 # limit of number of venues returned by Foursquare API
radius = 400 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WCO3PQS02C5V3FHVMAGTI5N2TNM5UBSKGZYQYRLOPEAGESXB&client_secret=MHHHHT2OT5TZFBMRVV4QK3NBX51RGSEKJTWLWBMNNX1LWYV4&v=20180605&ll=43.725882299999995,-79.31557159999998&radius=400&limit=80'

Getting the resultas of the exploring request

In [22]:
results = requests.get(url).json()

let's borrow the get_category_type function from the Foursquare lab.

In [23]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Extracting information from the JSON file and structing it intoa pandas dataframe

In [24]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Victoria Village Arena,Hockey Arena,43.723481,-79.315635
1,Portugril,Portuguese Restaurant,43.725819,-79.312785
2,Tim Hortons,Coffee Shop,43.725517,-79.313103
3,Eglinton Ave E & Sloane Ave/Bermondsey Rd,Intersection,43.726086,-79.313620


### Let's do the same process for all the North York neighborhoods

Creating a function that will help us executing these tasks

In [25]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Getting all the nearby venues of each neighborhood in North York and structuring them in a new data frame called North_York_venues

In [26]:
North_York_venues = getNearbyVenues(names=North_York_data['Neighborhood'],
                                   latitudes=North_York_data['Latitude'],
                                   longitudes=North_York_data['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [27]:
print('Total number of venues in North York : ', North_York_venues.shape[0])

Total number of venues in North York :  244


Number of venues per neighborhood

In [28]:
North_York_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",21,21,21,21,21,21
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",24,24,24,24,24,24
Don Mills,27,27,27,27,27,27
Downsview,15,15,15,15,15,15
"Fairview, Henry Farm, Oriole",67,67,67,67,67,67
Glencairn,4,4,4,4,4,4
Hillcrest Village,4,4,4,4,4,4
Humber Summit,4,4,4,4,4,4


#### Let's find out how many unique categories can be curated from all the returned venues

In [ ]:
print('There are {} uniques categories.'.format(len(North_York_venues['Venue Category'].unique())))

### Analysing the neighborhoods

In [ ]:
# one hot encoding
North_York_onehot = pd.get_dummies(North_York_venues[['Venue Category','Neighborhood']])

# add neighborhood column back to dataframe
North_York_onehot['Neighborhood'] = North_York_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [North_York_onehot.columns[-1]] + list(North_York_onehot.columns[:-1])
North_York_onehot = North_York_onehot[fixed_columns]

North_York_onehot

In [ ]:
from sklearn.cluster import KMeans

# set number of clusters
kclusters = 5

k_means = KMeans(init="k-means++", n_clusters=kclusters, n_init=12)
k_means.fit(North_York_onehot)
labels = k_means.labels_

print(labels)